# Annotation - Final Descriptors

In [39]:
import sys
sys.path.append('../data/') # TODO: it does not work

import subprocess
import os
from IPython.display import Video, VimeoVideo
import pandas as pd
import numpy as np
import time

In [1883]:
class Annotate:
    def __init__(self,file):
        self.df = pd.read_csv(file)
        self.annotation_done = []
        
    def get_labels(self):
        return(list(self.df.label_idx.unique()))

    def get_sample(self, label, size=100, seed=123):
        np.random.seed(seed)
        self.label = label
        df_sample = self.df[self.df.label_idx == label]
        self.video_list = np.random.choice(df_sample.M, size, False)
        videos = self.video_list
        return videos
    
    def pick_video(self, videos):
        n = np.random.randint(0,99)
        avi_path = videos[n]
        flag = os.path.exists(avi_path)
        #print(flag)
        while flag!=True:
            #print("GETTING ANOTHER ONE")
            flag = True
            avi_path, flag = self.pick_video(videos)
        return avi_path, flag        
    
    def play_video(self, avi_path, width=512, height=512):
        assert avi_path.split('.')[-1] == 'avi', "The input file {} is not avi extension".format(avi_path)
        mp4_path = avi_path.replace('.avi', '.mp4')
        cmds = ['ffmpeg', '-i', avi_path, mp4_path] 
        subprocess.Popen(cmds) # this line creates .mp4 file in the same directory

        time.sleep(0.5)

        video = Video(mp4_path, embed=True)

        return video
    
    def update_vals(self, row):
        data = self.descriptor_dict
        if row.M == data['M']:
            row.Both_Hands = data['Both_Hands']
            row.M_Out = data['M_Out']
            row.M_In = data['M_In']
            row.M_Up = data['M_Up']
            row.M_Down = data['M_Down']
            row.M_Front = data['M_Front']
            row.M_Back = data['M_Back']
            row.M_Iterative = data['M_Iterative']
            row.O_Out = data['O_Out']
            row.O_In = data['O_In']
            row.O_Up = data['O_Up']
            row.O_Down = data['O_Down']
            row.O_Front = data['O_Front']
            row.O_Back = data['O_Back']
            row.F_Thumb = data['F_Thumb']
            row.F_Index = data['F_Index']
            row.F_Middle = data['F_Middle']
            row.F_Ring = data['F_Ring']
            row.F_Pinky = data['F_Pinky']
        return row

    def update_annotation(self, descriptor_dict):
        self.descriptor_dict = descriptor_dict
        self.df = self.df.apply(self.update_vals, axis=1)
        print("Updated Annotations for video", descriptor_dict['M'])
        self.annotation_done.append(descriptor_dict['M'])
        
    def update_csv(self,file_name):
        self.df.to_csv(file_name, index=False)
    
    def create_descriptor_json(self,filename,method='mean'):
        df_exp = self.df.copy()
        df_exp = df_exp.dropna()
        descriptor_vals = {}
        for i in df_exp.label_idx.unique():
            if method == 'mean':
                descriptor_vals[str(i)] = dict(df_exp[df_exp.label_idx==i].iloc[:,3:22].mean())
            elif method == 'mode':
                descriptor_vals[str(i)] = dict(df_exp[df_exp.label_idx==19].iloc[:,3:22].astype('bool').mode().astype('int'))
            else:
                print("Invalid method choice")
                return 
        import json
        with open(filename, 'w') as fp:
            json.dump(descriptor_vals, fp)
            print("Successfully Created Json file of Descriptor Vector")
        return


CHANGE DIRECTORY TO THE DIRECTORY WHERE ALL YOUR VIDEO FILES ARE (edit cd command in next block). <br>
MAKE SURE TO PUT 'top50_videos_w_descriptors.csv' IN THE DIRECTORY AS WELL

In [41]:
cd /Users/kmanchel/Documents/COMPLETE_IsoGD_phase_1/`


/Users/kmanchel/Documents/COMPLETE_IsoGD_phase_1


In [1884]:
annotate = Annotate('annotated_kaushik_done5.csv')

In [1878]:
labels_ = annotate.get_labels()
labels_

[19,
 135,
 49,
 159,
 156,
 93,
 51,
 155,
 158,
 14,
 239,
 97,
 238,
 94,
 50,
 5,
 38,
 10,
 213,
 56,
 222,
 18,
 8,
 95,
 120,
 96,
 188,
 157,
 29,
 160,
 4,
 154,
 37,
 36,
 117,
 55,
 13,
 90,
 209,
 240,
 31,
 127,
 241,
 223,
 92,
 34,
 237,
 91,
 32,
 52]

Skipped: 49,51,155,50,213,56,55,34*
Note: Complicated gestures have very noisy training data for the purpose of classifying descriptors.

In [1866]:
videos = annotate.get_sample(label = 19)

Below line randomly picks one video from the selected class label above

In [1869]:
avi_path,_ = annotate.pick_video(videos)

In [1870]:
print("Playing video from directoy:",avi_path)
print("Gesture Label:", annotate.label)
annotate.play_video(avi_path)

Playing video from directoy: train/018/M_03493.avi
Gesture Label: 19


## ANNOTATION LABELS: 

| Label Name | Type   | Description
|------|------|------|
|   Both_Hands  | Hands | Are both hands used for gesture? 
|   M_Out  | Motion | Is Left or Right hand moving away from body? 
|   M_In  | Motion | Is Left or Right hand moving towards body?
|   M_Up  | Motion | Is Left or Right hand moving upwards?
|   M_Down  | Motion | Is Left or Right hand moving downwards?
|   M_Front  | Motion | Is Left or Right hand moving forwards?
|   M_Back  | Motion | Is Left or Right hand moving backwards?
|   M_Iterative  | Motion | Is Left or Right hand moving iterative/repetitively?
|   O_Out  | Orientation | Is Left or Right hand facing away from body?
|   O_In  | Orientation | Is Left or Right hand facing towards body?
|   O_Up  | Orientation | Is Left or Right hand facing upwards?
|   O_Down  | Orientation | Is Left or Right hand facing downwards?
|   O_Front  | Orientation | Is Left or Right hand facing forwards?
|   O_Back  | Orientation | Is Left or Right hand facing backwards?
|   F_Thumb  | Finger State | Is thumb sticking out?
|   F_Index  | Finger State | Is index finger sticking out?
|   F_Middle  | Finger State | Is middle finger sticking out?
|   F_Ring  | Finger State | Is ring finger sticking out?
|   F_Pinky  | Finger State | Is little pinky finger sticking out?


Annotations to be filled out in the dictionary below

In [1794]:
descriptors_annotated = {'M': avi_path,
                         'Both_Hands': 1,
                         'M_Out':1, 'M_In':1, 'M_Up':1, 'M_Down':0, 'M_Front':0, 'M_Back':0, 'M_Iterative':1,
                         'O_Out':0, 'O_In':0, 'O_Up':0, 'O_Down':1, 'O_Front':0, 'O_Back':0,
                         'F_Thumb':1, 'F_Index':1, 'F_Middle':1, 'F_Ring':1, 'F_Pinky':1} 


In [1795]:
#Annotations will be updated in the csv file based on descriptors_annotated dictionary
annotate.update_annotation(descriptors_annotated)

Updated Annotations for video train/027/M_05261.avi


In [1796]:
#List of annotations that are done
annotate.annotation_done

['train/045/M_08867.avi',
 'train/006/M_01077.avi',
 'train/017/M_03238.avi',
 'train/050/M_09927.avi',
 'train/172/M_34293.avi',
 'train/033/M_06594.avi',
 'train/165/M_32813.avi',
 'train/014/M_02654.avi',
 'train/029/M_05756.avi',
 'train/030/M_05984.avi',
 'train/017/M_03397.avi',
 'train/039/M_07736.avi',
 'train/030/M_05905.avi',
 'train/001/M_00009.avi',
 'train/036/M_07146.avi',
 'train/011/M_02076.avi',
 'train/031/M_06017.avi',
 'train/045/M_08904.avi',
 'train/168/M_33520.avi',
 'train/005/M_00895.avi',
 'train/167/M_33278.avi',
 'train/023/M_04456.avi',
 'train/014/M_02614.avi',
 'train/164/M_32708.avi',
 'train/165/M_32915.avi',
 'train/013/M_02413.avi',
 'train/047/M_09238.avi',
 'train/049/M_09679.avi',
 'train/171/M_34161.avi',
 'train/018/M_03517.avi',
 'train/026/M_05051.avi',
 'train/173/M_34536.avi',
 'train/020/M_03988.avi',
 'train/172/M_34319.avi',
 'train/176/M_35115.avi',
 'train/023/M_04566.avi',
 'train/041/M_08053.avi',
 'train/003/M_00429.avi',
 'train/021/

In [1797]:
#Checking how many annotations have been completed
len(annotate.annotation_done)

210

In [1875]:
#Write Annotations to new CSV 
file_name = 'annotated_kaushik_done5.csv'
annotate.update_csv(file_name)

In [1885]:
#Create json of descriptor for all labels. Mean and Mode methods available:
annotate.create_descriptor_json('annotated_kaushik_mean.json',method='mean')

Successfully Created Json file of Descriptor Vector
